# Human keyboard

In [ ]:
import gym
import keyboard
env = gym.make("MountainCar-v0", new_step_api=True, render_mode='human')
env._max_episode_steps = 100

# Play
state = env.reset()  # [position, velocity]
terminated = False # True if the player dies or wins
truncated = False # True if the time runs out
score = 0

while not terminated and not truncated:
    #Close window with ESC key
    if keyboard.is_pressed('esc'):
        env.close()
        assert False

    # Read keyboard input
    int_action = 1  # No push
    if keyboard.is_pressed('right'):
        int_action = 2  # right push
    elif keyboard.is_pressed('left'):
        int_action = 0  # left push

    # Send input to game
    state, reward, terminated, truncated, info = env.step(int_action)

    score += reward
print(score)
env.close()

# AI enviroment setup
run once before running the "AI test" and "AI train block"

In [ ]:
import gym
import neat
import os
import math
import pickle

def convert_to_int_action(action):
    int_action = 1  # No push

    if action[0] > 0.5:
        int_action = 2  # Right push
    elif action[1] > 0.5:
        int_action = 0  # Left push
    return int_action

# Getting file with values
config_path = os.path.abspath("config-feedforward.txt")
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
p = neat.Population(config)

p.add_reporter(neat.StdOutReporter(True))  # printing stats

# Train AI
Remember to run "AI enviroment setup" before 

In [ ]:
def ai_train(genomes, config):
    nets = []  # Neural networks/brain
    ge = []  # Genes

    # Setting up AI
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)  # Getting values from txt
        nets.append(net)  # Appending every Neural Network to array
        g.fitness = 0
        ge.append(g)  # Appending every gene to array.

    # training every agent
    for g, net in zip(ge, nets):
        # Start game
        state = env.reset()
        terminated = False
        truncated = False

        # Variables for calculating fitness
        high_vel = -math.inf
        low_vel = math.inf
        elapsed_steps = 0

        while not terminated and not truncated:
            action = net.activate((state))  # send state to AI

            int_action = convert_to_int_action(action)
            state, reward, terminated, truncated, info = env.step(int_action)
            g.fitness += reward
            pos, vel = state
            elapsed_steps += 1

            if vel > high_vel:
                high_vel = vel
            if vel < low_vel:
                low_vel = vel

# Environment setup
env = gym.make("MountainCar-v0", new_step_api=True)
env._max_episode_steps = 1000

num_training_generations = 10
winner = p.run(ai_train, num_training_generations)
print(winner)

# Save best gene in Pickle file
file = open('winner_neat.p', 'wb')
pickle.dump(winner, file)
file.close()

# Test AI
Remember to run "AI enviroment setup" before 

In [ ]:
import keyboard

def ai_test():
    # open pickle file
    file = open('winner_neat.p', 'rb')
    winner = pickle.load(file)
    file.close()
    print(winner)

    # Create Neural Network
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

    state = env.reset()
    terminated = False
    truncated = False
    score = 0

    while not terminated and not truncated:
        #Close window with ESC key
        if keyboard.is_pressed('esc'):
            env.close()
            assert False

        action = winner_net.activate((state))

        int_action = convert_to_int_action(action)
        state, reward, terminated, truncated, info = env.step(int_action)
        score += reward
    print(score)

# Environment setup
env = gym.make("MountainCar-v0", new_step_api=True, render_mode='human')
env._max_episode_steps = 1000
ai_test()
env.close()